# Multiples of 3 and 5

## バージョン情報

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx  
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, znver1)


## Problem 1
If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.
Find the sum of all the multiples of 3 or 5 below 1000.

> 1000 未満の 3 と 5 の倍数のすべての和を計算せよ。

## 方針またはプログラミング基礎
凝ったことをしていないのでPythonとほとんど同じように考えられる。
PythonとJuliaでプログラムを読みかえるのも大変だろうから、重複をいとわず書くことにしよう。

JuliaにはJuliaの都合があって、高速化を考えるといくつか工夫が必要だが、まずはあまり気にせず書くことにする。
### 方針1
まずは1000未満の3の倍数と5の倍数を全部作ってみよう。ただ、1000だと結果が見づらいので、`n=30`くらいにして見やすくする。

In [2]:
# 本来の値
N = 999

999

In [3]:
n = 30
threes = Vector{Int}(undef, 0)
for i in 1:n
    if i % 3 == 0 append!(threes, i) end
end

println(threes)

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]


In [4]:
fives = Vector{Int}(undef, 0)
for i in 1:n
    if i % 5 == 0 append!(fives, i) end
end

print(fives)

[5, 10, 15, 20, 25, 30]

このふたつのリストには重複がある。
具体的には15の倍数が重複する。
何らかの方法でこの重複を処理したい。

方法はいくつかある。
最終的には効率・速度も考えないといけないが、ここではそこまでは要求しない。

一番簡単なのはリストを一気に作る方法か？

In [5]:
numbers = Vector{Int}(undef, 0)
for i in 1:n
    if i % 3 == 0 || i % 5 == 0 append!(numbers, i) end
end
print(numbers)

[3, 5, 6, 9, 10, 12, 15, 18, 20, 21, 24, 25, 27, 30]

あとは和を取る。

In [6]:
numbers = Vector{Int}(undef, 0)
for i in 1:n
    if i % 3 == 0 || i % 5 == 0 append!(numbers, i) end
end
sum(numbers)

225

`n`を`N`に変えて計算してみよう。

In [7]:
numbers = Vector{Int}(undef, 0)
for i in 1:N
    if i % 3 == 0 || i % 5 == 0 append!(numbers, i) end
end
sum(numbers)

233168

これで確かに解答が得られた。
[解答1](#解答1)として別途まとめておこう。

### 方針2
引き続き効率やスピード度外視で考えよう。
今度はリストではなく集合で作ってみる。

まずは3の倍数から。

In [8]:
threes = Set{Int}()
for i in 1:n
    if i % 3 == 0 push!(threes, i) end
end
println(threes)

Set([6, 15, 21, 27, 18, 9, 12, 24, 30, 3])


変わったのは`threes = []`ではなく`set()`とした部分。
5の倍数も同じ。

In [9]:
fives = Set{Int}()
for i in 1:n
    if i % 5 == 0 push!(fives, i) end
end
print(fives)

Set([5, 15, 20, 25, 10, 30])

わざと同じ流れにしたが、はじめからリストの時と同じように`if i % 3 ==0 or i % 5 == 0`で作ってもいい。
あえて同じ流れにしたのは集合には便利な演算がありそれを紹介するため。
このデータ構造は重複があると勝手に重複を一意化してくれる。

特に和集合を作る演算があるので、それでリストの時と同じく、必要なすべての数をリストアップした`numbers`を作ってみよう。
具体的には関数`union`か`∪`演算子で和集合が作れる。

In [10]:
numbers1 = union(threes, fives)
numbers2 = ∪(threes, fives)
numbers3 = threes ∪ fives
println(numbers1 == numbers2)
println(numbers1 == numbers3)

true
true


In [11]:
sum(numbers3)

225

`n`の時の値は配列版と一致した。
`N`に変えれば正しい値が出るだろう。
まとめて[解答2](#解答2)にしておく。

## コードの単純化
アルゴリズムのレベルで他にもやりようはあるだろう。
ただパッと思いつかないので、とりあえずここではいくつかコードを簡素化してみる。
いったん全てリストで対応する。

### リスト内包表記
読みやすいかどうかといった問題もあるが、一般にJuliaは`for`で書くよりもリスト内包表記で書く方が速くメモリ効率もいいとされているようなので、リスト内包表記を使ってみよう。

まずは比較用にもともとの`for`で作ったリストを準備する。

In [12]:
threes1 = Vector{Int}(undef, 0)
for i in 1:n
    if i % 3 == 0 append!(threes1, i) end
end
print(threes1)

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]

In [13]:
threes2 = [i for i in 1:n if i % 3 == 0]
print(threes1 == threes2)

true

`for`で書くと4行だったのが1行になった。
慣れていないと読みづらいのは間違いない。
あと`for`の中身が長いとそもそも書けたものではない。

In [14]:
numbers = [i for i in 1:n if i % 3 == 0 || i % 5 == 0]
sum(numbers)

225

上のセルでは`n`で計算した。
本来の答え（リストの和）が欲しいだけならもっと短く書ける。
今度は`N`で計算しよう。

In [15]:
sum([i for i in 1:N if i % 3 == 0 || i % 5 == 0])

233168

### 高階関数
例えば`filter`を使ってみよう。
`filter`は第一引数が関数で、第二引数に「リスト」を取る関数で、第一引数の関数の真偽に応じてリストの要素をふるいにかける（フィルターする）。

よくわからなければ具体的に挙動を見るのが早い。
例を作って確認しよう。

In [16]:
function lt5(x)
    return x < 5
end

below1000 = 1:N
lessthan5 = filter(lt5, below1000)
println(lessthan5)

[1, 2, 3, 4]


`lt5`は less than 5 の頭文字で、5より小さい数に対して`True`を返す。
そして`filter`には`True`を返してほしい関数と対象リストを渡している。

ラムダの記法以外にも、Juliaは短い関数を簡単に書く記法があるのでまずはそれを紹介しよう。

In [17]:
lt5(x) = x < 5

below1000 = 1:N
lessthan5 = filter(lt5, below1000)
println(lessthan5)

[1, 2, 3, 4]


長い関数ならともかく、関数本体が短い場合、いちいち関数を定義するのが面倒な場合がある。
そういうときはラムダを使うといい。
これも例を見た方が早い。

In [18]:
below1000 = 1:N
lessthan5 = filter(x -> x < 5, below1000)
print(lessthan5)

[1, 2, 3, 4]

これを使って解答を作ってみよう。

In [19]:
all = 1:N
numbers = filter(x -> x % 3 == 0 || x % 5 == 0, all)
sum(numbers)

233168

このくらいのコードなら、次のようにもっとシンプルにしてもいいだろう。

In [20]:
sum(filter(x -> x % 3 == 0 || x % 5 == 0, 1:N))

233168

### パイプライン演算子
F#でも便利な演算子で、処理の流れが読みやすい。
単純な標準出力で比較してみよう。

In [21]:
println([1,2,3])
[1,2,3] |> println

[1, 2, 3]
[1, 2, 3]


関数が一つだけの場合は後者の方が面倒かもしれないが、`sum(filter(x -> x % 3 == 0 || x % 5 == 0, 1:999))`くらいの長さになってくるとパイプラインが読みやすくなる（ことがある）。
試してみよう。

In [22]:
div3or5(x) = x % 3 == 0 || x % 5 == 0
[1:999;] |> x -> filter(div3or5, x) |> sum

233168

もちろん`div3or5`もラムダにできる。

In [23]:
[1:N;] |> x -> filter(n -> n % 3 == 0 || n % 5 == 0, x) |> sum

233168

## 解答

### 解答1

In [24]:
numbers = Vector{Int}(undef, 0)
for i in 1:999
    if i % 3 == 0 || i % 5 == 0 append!(numbers, i) end
end
sum(numbers)

233168

### 解答2

In [25]:
threes = Set{Int}()
for i in 1:999
    if i % 3 == 0 push!(threes, i) end
end

fives = Set{Int}()
for i in 1:999
    if i % 5 == 0 push!(fives, i) end
end

numbers3 = threes ∪ fives
sum(numbers3)

233168

### 解答3

In [26]:
sum([i for i in 1:999 if i % 3 == 0 || i % 5 == 0])

233168

### 解答4

In [27]:
sum(filter(x -> x % 3 == 0 || x % 5 == 0, 1:999))

233168

### 解答5

In [28]:
[1:999;] |> x -> filter(n -> n % 3 == 0 || n % 5 == 0, x) |> sum

233168